In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from PIL.Image import core as image
import os
from PIL import Image

Using TensorFlow backend.


In [3]:
# Run through image files and determine if any are the wrong shape
# Count images by crop/disease type
data_path = 'PlantVillage-Dataset/raw_image_data/color'
diff_shape_count = 0
img_count = 0
leaf_type_img_count = 0
leaf_type_img_count_list = []
for folder in os.listdir(data_path):
    for image in os.listdir(f'{data_path}/{folder}'):
        img_loc = f'{data_path}/{folder}/{image}'
        img = Image.open(img_loc)
        arr = np.array(img)
        img_shape = arr.shape
        img_count += 1
        leaf_type_img_count += 1
        if img_shape != (256, 256, 3):
            diff_shape_count += 1
            print(img_loc)
            print(img_shape)
        else:
            continue
    leaf_type_img_count_list.append((folder, leaf_type_img_count))
    leaf_type_img_count = 0
print(diff_shape_count)
print(img_count)
leaf_type_img_count_list

PlantVillage-Dataset/raw_image_data/color/Pepper,_bell___healthy/42f083e2-272d-4f83-ad9a-573ee90e50ec___Screen Shot 2015-05-06 at 4.01.13 PM.png
(256, 256, 4)
1
54305


[('Strawberry___healthy', 456),
 ('Grape___Black_rot', 1180),
 ('Potato___Early_blight', 1000),
 ('Blueberry___healthy', 1502),
 ('Corn_(maize)___healthy', 1162),
 ('Tomato___Target_Spot', 1404),
 ('Peach___healthy', 360),
 ('Potato___Late_blight', 1000),
 ('Tomato___Late_blight', 1909),
 ('Tomato___Tomato_mosaic_virus', 373),
 ('Pepper,_bell___healthy', 1478),
 ('Orange___Haunglongbing_(Citrus_greening)', 5507),
 ('Tomato___Leaf_Mold', 952),
 ('Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 1076),
 ('Cherry_(including_sour)___Powdery_mildew', 1052),
 ('Apple___Cedar_apple_rust', 275),
 ('Tomato___Bacterial_spot', 2127),
 ('Grape___healthy', 423),
 ('Tomato___Early_blight', 1000),
 ('Corn_(maize)___Common_rust_', 1192),
 ('Grape___Esca_(Black_Measles)', 1383),
 ('Raspberry___healthy', 371),
 ('Tomato___healthy', 1591),
 ('Cherry_(including_sour)___healthy', 854),
 ('Tomato___Tomato_Yellow_Leaf_Curl_Virus', 5357),
 ('Apple___Apple_scab', 630),
 ('Corn_(maize)___Northern_Leaf_Blight', 985)

In [ ]:
epochs = 25
init_lr = 1e-3
bs = 32
default_image_size = (256, 256)
image_size = 0
directory_root = '../input/plantvillage/'
width=256
height=256
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None